In [1]:
from Bio.PDB.PDBParser import PDBParser

In [2]:
parser = PDBParser()

In [3]:
struc = parser.get_structure("1", "./data/alanine_dipeptide/1.pdb")

/Users/janik/.conda/envs/py/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 22
  warnings.warn(


In [6]:
residues = list(struc.get_residues())
residues

[<Residue ALA het=  resseq=1 icode= >, <Residue ALA het=  resseq=2 icode= >]

In [7]:
res = residues[0]

In [8]:
res

<Residue ALA het=  resseq=1 icode= >

In [11]:
atoms = list(res.get_atoms())

In [13]:
atoms

[<Atom N>,
 <Atom CA>,
 <Atom C>,
 <Atom O>,
 <Atom CB>,
 <Atom H>,
 <Atom HA>,
 <Atom 1HB>,
 <Atom 2HB>,
 <Atom 3HB>]

In [15]:
a = atoms[0]

In [17]:
v = a.get_vector()

In [21]:
v.angle(atoms[1].get_vector())

1.3475739258740307

In [ ]:
v.